In [1]:
## Standard imports
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import os
import json
import torch
import astroddpm
import tqdm.notebook as tqdm

## Relative imports
from astroddpm.runners import Diffuser, config_from_id, get_samples
from astroddpm.analysis.validationMetrics import powerSpectrum, minkowskiFunctional, basics
from quantimpy import morphology as mp
from quantimpy import minkowski as mk
from astroddpm.analysis import overfitting_check
from astroddpm.utils.plot import check_nearest_epoch, plot_losses, check_training_samples, plot_comparaison
from astroddpm.diffusion.dm import DiscreteSBM
from astroddpm.diffusion.stochastic.sde import DiscreteVPSDE
from astroddpm.diffusion.models.network import ResUNet
import astroddpm.utils.colormap_custom 

In [2]:
save_fig = True

params = {'text.usetex': True,
         'text.latex.preamble': r'\usepackage{amsmath}',
         "font.family": "serif", "font.serif": ["Computer Modern Roman"],
         'font.size': 12,
         'image.interpolation': 'none'}
plt.rcParams.update(params)

In [10]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

MODEL_ID = 'DiscreteSBM_MultiSigmaVPSDE_I_BPROJ_N_1000_bottleneck_16_firstc_6'

In [11]:
placeholder_dm = DiscreteSBM(DiscreteVPSDE(1000), ResUNet())

diffuser= Diffuser(placeholder_dm)
diffuser.load(config=config_from_id(MODEL_ID), also_ckpt=True, for_training=True)

No model id found
Loading the diffuser from a config dict.
Loading the optimizer and scheduler states from the checkpoint, as well as previous training info. You should change the model id if you want to finetune the model.


In [24]:
if diffuser.diffmodel.has_thetas:
    theta_mean = diffuser.diffmodel.ps.sample_theta(10000).mean(dim=0, keepdim=True).to(device)
    theta_mean

In [25]:
image = diffuser.test_dataloader.dataset[0].unsqueeze(0).unsqueeze(1).to(device)
image.shape

torch.Size([1, 1, 256, 256])

In [26]:
NUM_SAMPLES = 64
NOISE_LEVEL = 0.1

In [27]:
all_timesteps = torch.tensor(range(diffuser.diffmodel.sde.N))
all_noise_levels = diffuser.diffmodel.sde.noise_level(all_timesteps).reshape(-1)

In [28]:
closest_timestep = torch.argmin(torch.abs(all_noise_levels - NOISE_LEVEL))

In [29]:
closest_timestep, diffuser.diffmodel.sde.noise_level(closest_timestep).reshape(-1)

(tensor(26, device='cuda:0'), tensor([0.1000], device='cuda:0'))

### No dependence on $\phi$

### Depend on $\phi$